In [6]:
import numpy

# generate cartesian product of multiple array
# input: tuple of arrays
def cartesian_product(arrays):
    broadcastable = numpy.ix_(*arrays)
    broadcasted = numpy.broadcast_arrays(*broadcastable)
    rows, cols = reduce(numpy.multiply, broadcasted[0].shape), len(broadcasted)
    out = numpy.empty(rows * cols, dtype=broadcasted[0].dtype)
    start, end = 0, rows
    for a in broadcasted:
        out[start:end] = a.reshape(-1)
        start, end = end, end + rows
    return out.reshape(cols, rows).T

In [7]:
def createSampleWavepacket(D, shape):
    packet = HagedornWavepacket(D,1,0.9)
    
    # create sample parameters
    p = numpy.zeros((D))
    q = numpy.zeros((D))
    Q = numpy.eye(D, dtype=complex)
    P = 1j*numpy.eye(D, dtype=complex)
    for i in range(0,D):
        q[i] = math.cos(i+1)
        p[i] = math.sin(i+1)
        for j in range(0,D):
            Q[i,j] += 0.3*math.sin(i+j+1) + 0.3j*math.cos(i+j+1)
            P[i,j] += 0.3*math.cos(i+j+1) + 0.3j*math.sin(i+j+1)
    packet.set_parameters( (q,p,Q,P), key=("q","p","Q","P"))
    
    # set shape
    packet.set_basis_shapes((shape),0)
    
    # create sample coefficients
    for node in shape.get_node_iterator():
        falloff = 0.1

        x = 0.0
        y = 0.0
        s = 0
        for d in range(0,D):
            x += math.sin(node[d] + 1.0*(d+1)/D)
            y += math.cos(node[d] + 1.5*(d+1)/D)
            s += node[d]

        x *= math.exp(-falloff*s)
        y *= math.exp(-falloff*s)

        packet.set_coefficient(0, node, x + y*1j)
    
    return packet

In [8]:
import numpy
import numpy.random as rand
import math
from WaveBlocksND import *

D = 3

shape = HyperCubicShape((15,15,15))

packet = createSampleWavepacket(D, shape)

x0 = numpy.zeros((D,1))
for d in range(0,D):
    x0[d,0] = (d+1)/float(2*D)
print "Selected Evaluation: ", packet.slim_recursion(x0,0)

rand.seed(0)
points = rand.randn(D,20)

results = packet.slim_recursion(points,0)[0]

import csv
with open("wavepacket.csv", "w") as filehandle:
    csvfile = csv.writer(filehandle, delimiter=' ')
    for point,result in zip(points.T,results):
        csvfile.writerow(tuple(point) + (numpy.real(result),) + (numpy.imag(result),))
print "file written"

    #x = numpy.linspace(-5.0,5.0,21)
    #y = numpy.linspace(-5.0,5.0,21)
    #grid = cartesian_product((x,y)).T

    #values = packet.slim_recursion(grid,0)[0]

    #csv = zip(grid[0,:], grid[1,:], numpy.real(values), numpy.imag(values))
    #numpy.savetxt("wavepacket.csv", csv, fmt=["%f","%f","%f","%f"])



Selected Evaluation:  [[-157.01932362-307.03955789j]]
file written
